In [1]:
from github import Github
import pandas as pd

from repository import Repository

In [2]:
# Read Token
access_token = open('GITHUB_TOKEN.txt', 'r').read()
user = Github(access_token)
user.per_page = 100

In [3]:
df = pd.read_json ('./data/python_best.json')

In [4]:
a = df['id'][0]

repo = user.get_repo(int(a))

In [5]:
repo.clone_url

'https://github.com/ytdl-org/youtube-dl.git'

In [6]:
r = Repository(repo, './repos/'+repo.name)

In [7]:
r.download()

Cloned Repository:  youtube-dl


In [8]:
import pygount

In [ ]:
analysis = pygount.source_analysis(f, repo['language'], encoding = 'utf-8')

In [11]:
repo = user.get_repo("PyGithub/PyGithub")
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    else:
        print(file_content)

ContentFile(path=".flake8")
ContentFile(path=".git-blame-ignore-revs")
ContentFile(path=".gitignore")
ContentFile(path=".isort.cfg")
ContentFile(path=".travis.yml")
ContentFile(path="CONTRIBUTING.md")
ContentFile(path="COPYING")
ContentFile(path="COPYING.LESSER")
ContentFile(path="DEPLOY.md")
ContentFile(path="MAINTAINERS")
ContentFile(path="MANIFEST.in")
ContentFile(path="README.md")
ContentFile(path="manage.sh")
ContentFile(path="requirements.txt")
ContentFile(path="setup.py")
ContentFile(path="test-requirements.txt")
ContentFile(path=".github/stale.yml")
ContentFile(path="doc/.gitignore")
ContentFile(path="doc/Design.md")
ContentFile(path="doc/changes.rst")
ContentFile(path="doc/conf.py")
ContentFile(path="doc/examples.rst")
ContentFile(path="doc/github.rst")
ContentFile(path="doc/index.rst")
ContentFile(path="doc/introduction.rst")
ContentFile(path="doc/reference.rst")
ContentFile(path="doc/utilities.rst")
ContentFile(path="github/AuthenticatedUser.py")
ContentFile(path="github/Aut

In [13]:
import requests
r = requests.get('https://github.com/kennethreitz/requests/blob/master/README.rst')